In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Now I am going to add BatchNormalization for increasing the accuracy

Reduce the parameters to 11k

Now addding the dropout for increasing the accuracy

In this script I have Global Average Pooling to decrease the parameters

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop1 = nn.Dropout(0.15)
        self.conv1 = nn.Conv2d(1, 10, 3, padding=0) #input=28 Output=26
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 10, 3, padding=0) #input=26 Output=24
        self.bn2 = nn.BatchNorm2d(10)
        self.conv3 = nn.Conv2d(10, 20, 3, padding=0) #input=24 Output=22
        self.bn3 = nn.BatchNorm2d(20)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(20, 10, 1, padding=0) #input=22 Output=11
        self.bn4 = nn.BatchNorm2d(10)
        self.conv5 = nn.Conv2d(10, 10, 3, padding=0) #input=11 Output=09
        self.bn5 = nn.BatchNorm2d(10)
        self.conv6 = nn.Conv2d(10, 20, 3, padding=0) #input=09 Output=07
        self.bn6 = nn.BatchNorm2d(20)
        self.conv7 = nn.Conv2d(20, 10, 1, padding=0) #input=07 Output=07
        self.bn7 = nn.BatchNorm2d(10)
        self.gap = nn.AvgPool2d(kernel_size = 7) # ADDED GAP

    def forward(self, x):
        x = self.pool1(self.drop1(F.relu(self.bn3(self.conv3(self.drop1(F.relu(self.bn2(self.conv2(self.drop1(F.relu(self.bn1(self.conv1(x)))))))))))))
        x = self.drop1(F.relu(self.bn5(self.conv5(self.drop1(F.relu(self.bn4(self.conv4(x))))))))
        x = self.drop1(F.relu(self.bn7(self.conv7(self.drop1(F.relu(self.bn6(self.conv6(x))))))))
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
           Dropout-3           [-1, 10, 26, 26]               0
            Conv2d-4           [-1, 10, 24, 24]             910
       BatchNorm2d-5           [-1, 10, 24, 24]              20
           Dropout-6           [-1, 10, 24, 24]               0
            Conv2d-7           [-1, 20, 22, 22]           1,820
       BatchNorm2d-8           [-1, 20, 22, 22]              40
           Dropout-9           [-1, 20, 22, 22]               0
        MaxPool2d-10           [-1, 20, 11, 11]               0
           Conv2d-11           [-1, 10, 11, 11]             210
      BatchNorm2d-12           [-1, 10, 11, 11]              20
          Dropout-13           [-1, 10, 11, 11]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...



Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 16):
    print("Epochs", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

Epochs 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.5475, Accuracy: 8787/10000 (88%)

Epochs 2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0745, Accuracy: 9792/10000 (98%)

Epochs 14


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0712, Accuracy: 9804/10000 (98%)

Epochs 15


loss=0.09835606813430786 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 40.04it/s]



Test set: Average loss: 0.0787, Accuracy: 9787/10000 (98%)

